In [51]:
#Hyperparameter tuning
import random
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split

df=pd.read_csv("D:\\AppStoneLab\\Day 2\\Iris.csv")

X = df.drop(columns=['Id', 'Species'], axis=1)
y = df['Species'].map({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

parameter_space = {
    "max_depth": [1,2,3,4,5],
    "criterion": ["gini", "entropy", "log_loss"],
    "splitter": ["best", "random"],
    "max_features": np.linspace(1,9,1)
}

DT=DecisionTreeClassifier()
DT_RCV=RandomizedSearchCV(DT, parameter_space)
DT_RCV.fit(X_train,y_train)

print("Best parameters through Randomized search: ", DT_RCV.best_params_)
print("Best score for Randomized search: ", DT_RCV.best_score_)

DT_GCV=GridSearchCV(DT, parameter_space)
DT_GCV.fit(X_train,y_train)

print("Best parameter through Grid search: ", DT_GCV.best_params_)
print("Best score for Grid search: ", DT_GCV.best_score_)

Best parameters through Randomized search:  {'splitter': 'best', 'max_features': np.float64(1.0), 'max_depth': 4, 'criterion': 'entropy'}
Best score for Randomized search:  0.9735177865612649
Best parameter through Grid search:  {'criterion': 'gini', 'max_depth': 4, 'max_features': np.float64(1.0), 'splitter': 'best'}
Best score for Grid search:  0.9735177865612649
